In [ ]:
import re

def heuristic_split(full):
    if not isinstance(full, str):
        return None, None

    text = full.strip()

    # Якщо весь текст у лапках
    if re.match(r'^[\"«“„].+[\"»”]$', text):
        clean_name = text.strip('«»“”"')
        return None, clean_name

    # Спроба: розбити по першій лапці
    m = re.search(r'[\"«“„]', text)
    if m:
        opf = text[:m.start()].strip()
        name = text[m.start():].strip('«»“”" ')
        if opf:
            return opf, name

    # Спроба: знайти перші слова верхнього регістру (OPF)
    parts = text.split()
    pos = 0
    for i, w in enumerate(parts):
        if not w.isupper() and not w.istitle():
            pos = i
            break

    opf = " ".join(parts[:pos]).strip() or None
    name = " ".join(parts[pos:]).strip()

    return opf, name

def extract_opf_and_name(full_name):
    if not isinstance(full_name, str):
        return None, None
    
    text = full_name.strip()
    text_lower = text.lower()

    # 1) Спроба знайти OPF у офіційному Excel-словнику
    for opf in opf_list:
        if opf in text_lower:
            # знайдена OPF
            opf_original = next(
                (x for x in opf_df["FIRM_OPFNM"] if x.lower().strip() == opf),
                opf
            )

            # Видаляємо OPF з назви
            name_clean = text_lower.replace(opf, "").strip('«»“”"').strip()

            # Якщо ім'я вийшло порожнім → fallback
            if not name_clean:
                return heuristic_split(full_name)

            return opf_original, name_clean

    # 2) Якщо нічого не знайдено → fallback
    return heuristic_split(full_name)

df["OPF"], df["FIRM_NAME"] = zip(*df["Повна назва"].apply(extract_opf_and_name))



## Особи

In [ ]:
import ast
import pandas as pd

def parse_list_column(row_value):
    """Перетворює рядок зі списком dict у Python-список."""
    if not isinstance(row_value, str):
        return []
    try:
        return ast.literal_eval(row_value)
    except:
        return []


# -------------------------------
# 1) УПОВНОВАЖЕНІ ОСОБИ
# -------------------------------
def expand_authorized(df):
    max_items = 5
    out = {}

    for idx, row in df["Уповноважені особи"].items():
        people = parse_list_column(row)
        record = {}

        for i, p in enumerate(people[:max_items], start=1):
            record[f"Authorized_{i}_Name"] = p.get("ПІБ")
            record[f"Authorized_{i}_Role"] = p.get("Роль")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


# -------------------------------
# 2) ЗАСНОВНИКИ
# -------------------------------
def expand_founders(df):
    max_items = 5
    out = {}

    for idx, row in df["Засновники"].items():
        founders = parse_list_column(row)
        record = {}

        for i, f in enumerate(founders[:max_items], start=1):
            record[f"Founder_{i}_Name"] = f.get("ПІБ / Назва")
            record[f"Founder_{i}_Country"] = f.get("Країна")
            record[f"Founder_{i}_Contribution"] = f.get("Розмір внеску")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


# -------------------------------
# 3) БЕНЕФІЦІАРИ
# -------------------------------
def expand_beneficiaries(df):
    max_items = 5
    out = {}

    for idx, row in df["Бенефіціари"].items():
        bens = parse_list_column(row)
        record = {}

        for i, b in enumerate(bens[:max_items], start=1):
            record[f"Benef_{i}_Name"] = b.get("ПІБ")
            record[f"Benef_{i}_Country"] = b.get("Країна")
            record[f"Benef_{i}_Share"] = b.get("Частка")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


In [ ]:
df_auth = expand_authorized(df)
df_found = expand_founders(df)
df_benef = expand_beneficiaries(df)

df_final = pd.concat([df, df_auth, df_found, df_benef], axis=1)

df_final = df_final.drop(columns=[
    "Уповноважені особи",
    "Засновники",
    "Бенефіціари"
], errors="ignore")


# -

In [ ]:
# ------------------------
# Бенефіціари
# ------------------------

def expand_beneficiary(df: pd.DataFrame, )